In [7]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver


In [8]:
driver=webdriver.Chrome()

In [9]:
url='https://www.amazon.com'
driver.get(url)

In [12]:
def get_url(search_item):
    template='https://www.amazon.com/s?k={}&ref=nb_sb_noss_2'
    search_item=search_item.replace(' ','+')
    return template.format(search_item)


In [14]:
url=get_url('ultrawide monitor')
print(url)

https://www.amazon.com/s?k=ultrawide+monitor&ref=nb_sb_noss_2


In [15]:
driver.get(url)

# Extract the collection


In [18]:
soup=BeautifulSoup(driver.page_source,'html.parser')

In [24]:
results=soup.find_all('div',{'data-component-type':'s-search-result'})

In [25]:
print(len(results))

22


# Prototype the record


In [26]:
item=results[0]

In [28]:
print(item.prettify())

<div class="s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 AdHolder sg-col sg-col-12-of-16" data-asin="B078GSH1LV" data-cel-widget="search_result_0" data-component-id="34" data-component-type="s-search-result" data-index="0" data-uuid="33d63f64-07fb-4565-bb90-14cdd2554706">
 <div class="sg-col-inner">
  <span cel_widget_id="MAIN-SEARCH_RESULTS-0" class="celwidget slot=MAIN template=SEARCH_RESULTS widgetId=search-results" data-cel-widget="MAIN-SEARCH_RESULTS-0" data-csa-c-id="mkf8ew-8yxrbn-nl220n-52wgr4">
   <div class="rush-component" data-component-id="35" data-component-props='{"percentageShownToFire":"50","batchable":true,"requiredElementSelector":".s-image","url":"https://www.amazon.com/gp/sponsored-products/logging/log-action.html?qualifier=1618717312&amp;id=573862076904716&amp;widgetName=sp_atf&amp;adId=200009857000351&amp;eventType=1&amp;adIndex=0"}' data-component-type="s-impression-logger">
    <div class="rush-component" data-component-id="36" data-component-type="sp-spo

In [29]:
atag=item.h2.a
print(atag)

<a class="a-link-normal a-text-normal" href="/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&amp;adId=A102645020ZZO08BEH16G&amp;url=%2FLG-34WK650-W-34-UltraWide-21%2Fdp%2FB078GSH1LV%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dultrawide%2Bmonitor%26qid%3D1618717312%26sr%3D8-1-spons%26psc%3D1&amp;qualifier=1618717312&amp;id=573862076904716&amp;widgetName=sp_atf">
<span class="a-size-medium a-color-base a-text-normal" dir="auto">LG 34WK650-W 34" UltraWide 21:9 IPS Monitor with HDR10 and FreeSync (2018), Black/White</span>
</a>


In [32]:
description=atag.text.strip()

In [35]:
url=atag.get('href')

In [40]:
price_parent=item.find('span','a-offscreen').text.strip()
price_parent

'$346.99'

In [41]:
reviews=item.i.text
reviews

'4.7 out of 5 stars'

In [43]:
review_count=item.find('span',{'class':'a-size-base','dir':'auto'}).text
review_count

'950'

In [52]:
# Genrealize the pattern
def extract_record(item):
    atag=item.h2.a
    description=atag.text.strip()
    url=atag.get('href')
    price_parent=item.find('span','a-price')
    price=price_parent.find('span','a-offscreen').text
    reviews=item.i.text
    review_count=item.find('span',{'class':'a-size-base','dir':'auto'}).text
        
    result=(description,price,reviews,review_count,url)
    return result

In [ ]:
records=[]
results=soup.find_all('div',{'data-component-type':'s-search-result'})
for item in results:
    records.append(extract_record(item))

# Error Handling

In [56]:
# Genrealize the pattern
def extract_record(item):
    atag=item.h2.a
    description=atag.text.strip()
    url='https://www.amazon.com'+atag.get('href')
    
    try:

        price_parent=item.find('span','a-price')
        price=price_parent.find('span','a-offscreen').text
    except AttributeError:
        return 
    
    try:
        reviews=item.i.text
        review_count=item.find('span',{'class':'a-size-base','dir':'auto'}).text
    except AttributeError:
        reviews=''
        review_count=''

    result=(description,price,reviews,review_count,url)
    return result

In [61]:
records=[]
results=soup.find_all('div',{'data-component-type':'s-search-result'})
for item in results:
    record=extract_record(item)
    if record:
        records.append(record)

In [62]:
record

('Lenovo G34w-10 34-Inch WQHD Curved Gaming Monitor, 21:9 Ultra-Wide, 3 Side NearEdgeless, 3440 x 1440, AMD FreeSync, 144Hz, HDMI, DP, Adjustable Stand, VESA, 66A1GCCBUS, Black',
 '$422.07',
 '4.2 out of 5 stars',
 '48',
 'https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_btf_aps_sr_pg1_1?ie=UTF8&adId=A01469583JZSMOW3XCVVU&url=%2FLenovo-G34w-10-34-Inch-Curved-Monitor%2Fdp%2FB084R2XYM9%2Fref%3Dsr_1_22_sspa%3Fdchild%3D1%26keywords%3Dultrawide%2Bmonitor%26qid%3D1618717312%26sr%3D8-22-spons%26psc%3D1&qualifier=1618717312&id=573862076904716&widgetName=sp_btf')

In [63]:
for row in records:
    print(row[1])

$346.99
$199.99
$429.97
$599.99
$1,496.99
$149.99
$226.99
$209.99
$110.99
$445.88
$319.99
$346.99
$329.99
$30.95
$346.99
$59.66
$59.99
$422.07


# Getting next page


In [64]:
def get_url(search_item):
    template='https://www.amazon.com/s?k={}&ref=nb_sb_noss_2'
    search_item=search_item.replace(' ','+')
    
    url=template.format(search_item)
    url+='&page{}'
    return url


# Putting all Together

In [69]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver


def get_url(search_item):
    template='https://www.amazon.com/s?k={}&ref=nb_sb_noss_2'
    search_item=search_item.replace(' ','+')
    
    url=template.format(search_item)
    url+='&page{}'
    return url

# Genrealize the pattern
def extract_record(item):
    atag=item.h2.a
    description=atag.text.strip()
    url='https://www.amazon.com'+atag.get('href')
    
    try:

        price_parent=item.find('span','a-price')
        price=price_parent.find('span','a-offscreen').text
    except AttributeError:
        return 
    
    try:
        reviews=item.i.text
        review_count=item.find('span',{'class':'a-size-base','dir':'auto'}).text
    except AttributeError:
        reviews=''
        review_count=''

    result=(description,price,reviews,review_count,url)
    return result

def main(search_term):
    driver=webdriver.Chrome()
    url='https://www.amazon.com'
    record=[]
    url=get_url(search_term)
    for page in range(1,21):
        driver.get(url.format(page))
        soup=BeautifulSoup(driver.page_source,'html.parser')
        results=soup.find_all('div',{'data-component-type':'s-search-result'})
        
        for item in results:
            record=extract_record(item)
            if record:
                records.append(record)
    driver.close()
    
    with open('Amazon.csv','w',newline='',encoding='utf-8') as f:
        writer=csv.writer(f)
        writer.writerow(['Description','Price','Rating','ReviewCount','Url'])
        writer.writerows(records)
        

In [70]:
main('ultrawide monitor')